# Miller Op Amp Sizing

In this notebook the models trained in `model_training.ipynb` are used to size a Miller OpAmp shown in the following figure for two technologies as described in the paper and presentation.

![Miller Operational Amplifier](./fig/moa.png)

In [1]:
%matplotlib inline

In [2]:
import os
import torch
import numpy as np
import torch as pt
import pandas as pd
import joblib as jl
from functools import partial
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, minmax_scale

from precept import PreceptModule

In [ ]:
class PrimitiveDevice():
    def __init__( self, model_path
                , scale_x_path, scale_y_path
                , params_x, params_y ):
        
        self.params_x = params_x
        self.params_y = params_y
        
        self.model = 
        
    def predict(X):